In [ ]:
db_connection = sql.connect(
    host="alifshop-mysql-2.alifshop.tj",
    user="your_user",
    password="your_password",
    auth_plugin="mysql_native_password",
)
application = pd.read_sql_query(
    f"""
select date_format(date_add(a.created_at, interval 5 hour), '%Y-%m-%d')  date,
       date_add(a.created_at, interval 5 hour) created_at,
       case
        when type_id = 1 then 'salom'
        when type_id = 2 then  'cash'
        end as a_type,
       a.id application_id,
       p.name product,
       concat('https://alifshop.tj/product/', p.slug) product_link,
       s.name shop,
       c2.name category,
       pc2.name parent_category,
       (ii.price-ii.discount)*ii.quantity as final_price,
       ii.discount,
       case
        when ii.discount >0 then 'По скидке'
        else 'Простая'
       end AS promotion,
       c.name city,
       a.gid crm_id,
       case
        when a.source = 'partner'  then 'Partner'
        else 'Client'
        end as created_by,
        partner_confirmation,
        crm_status_id,
        a.status as alifshop_status
from service_application.applications a
left join service_application.invoice_items ii on a.id = ii.application_id
left join service_merchant.shops s on a.shop_id = s.id
left join service_product.offers o on ii.offer_id = o.id
left join service_product.products p on o.product_id = p.id
left join service_setting.cities c on o.city_id = c.id
left join service_product.product_categories pc on p.id = pc.product_id
left join service_product.categories c2 on pc.category_id = c2.id
left join service_product.categories pc2 on c2.parent_id =  pc2.id
where date_add(a.created_at, interval 5 hour) >= '2023-10-01' and pc.is_default = 1 and a.gid is not null and s.name not like '%Electro Car%'
 """,
    db_connection,
)
application.crm_id = application.crm_id.astype("int32")

# ---------------------------------------------

db_connection_crm = sql.connect(
    host="192.168.15.26",
    database="crm_alif_tj",
    user="your_user",
    password="your_password",
    auth_plugin="mysql_native_password",
)
application_crm = pd.read_sql_query(
    f""" 
SELECT a.id, a.application_status_id, `as`.display_name as status, a.client_id, ca.created_at, status_changed_at,
a.purpose, a.amount
from credit_applications ca
left join applications a on ca.application_id = a.id
left join application_statuses `as` on a.application_status_id = `as`.id
where date_format(ca.created_at, '%Y-%m-%d') >= '2023-10-01' and (a.purpose like '%alifshop%' or a.purpose like '%alif shop%' or a.purpose like '%alif partners%')
""",
    db_connection_crm,
)
data_new = application.merge(
    application_crm,
    left_on="crm_id",
    right_on="id",
    how="left",
    suffixes=("_alifshop", "_crm"),
)[
    [
        "date",
        "created_at_alifshop",
        "a_type",
        "application_id",
        "product",
        "product_link",
        "shop",
        "category",
        "parent_category",
        "final_price",
        "discount",
        "promotion",
        "city",
        "crm_id",
        "created_by",
        "status",
        "client_id",
        "created_at_crm",
        "status_changed_at",
        "purpose",
        "amount",
    ]
]
print(data_new.shape)
data_new.crm_id = data_new.crm_id.astype("str")
pd_gbq.to_gbq(data_new, table_id, project_id=project_id, if_exists="replace")

In [7]:
#1. В колоде 52 карты. Мы решили вытащить из неё одну карту. Какова вероятность того, что это будет туз?
total_cards = 52
aces = 4
probability_ace = aces / total_cards
print(f"Вероятность вытащить туза: {probability_ace:.4f}")

Вероятность вытащить туза: 0.0769


In [9]:
#2. Какова вероятность того что у трёх случайно выбранных людей будет одинаковая группа крови? Подсказка: для решения этой задачи нужно использовать как правило умножения так и правило сложения.
p_A = 0.40
p_B = 0.11
p_AB = 0.04
p_O = 0.45


p_same = p_A**3 + p_B**3 + p_AB**3 + p_O**3

print(f"Вероятность, что у трёх людей одинаковая группа крови: {p_same:.4f}")


Вероятность, что у трёх людей одинаковая группа крови: 0.1565


In [11]:
#3. Студент, сдаёт тест состоящий из 30 вопросов с четырьмя вариантами ответов, из которых лишь один правильный. Так как он вообще не посещал лекции, он просто наугад выбирает один из вариантов для каждого вопроса. Какова вероятность что он правильно ответит хотя бы на один вопрос?
p_correct = 1/4
p_wrong = 3/4
questions = 30

p_all_wrong = (p_wrong)**questions
p_at_least_one = 1 - p_all_wrong

print(f"Вероятность правильно ответить хотя бы на один вопрос: {p_at_least_one:.6f}")


Вероятность правильно ответить хотя бы на один вопрос: 0.999821


In [ ]:
#4. Какова вероятность что хотя бы у одного из двух случайно выбранных людей месяц рождения такой же как и у вас?
p_match = 1/12
p_no_match = 11/12

p_both_no_match = p_no_match**2
p_at_least_one = 1 - p_both_no_match

print(f"Вероятность что хотя бы у одного из двух людей месяц рождения такой же как и у вас: {p_at_least_one:.4f}")


Вероятность что хотя бы у одного из двух людей месяц рождения такой же как и у вас: 0.1597


In [ ]:
#5.В колоде 52 карты. Мы решили вытащить из неё одну карту. Какова вероятность того что это будет либо валет, либо пиковая карта?
total_cards = 52
jacks = 4
spades = 13
jack_of_spades = 1
favorable_outcomes = jacks + spades - jack_of_spades
probability = favorable_outcomes / total_cards
print(f"Вероятность вытащить либо валет, либо пиковую карту: {probability:.4f}")

Вероятность вытащить либо валет, либо пиковую карту: 0.3077
